In [40]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
cd ..

C:\Projects\python\recommender


In [2]:
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

from utils import get_log_dir

In [3]:
from datasets import SeqTopcoder
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM, TransProbFM
from models.fm_learner import simple_loss, trans_loss, simple_weight_loss, trans_weight_loss

In [4]:
DEVICE = T.cuda.current_device()
BATCH = 2000
SHUFFLE = True
WORKERS = 0
NEG_SAMPLE = 5
REGS_PATH = Path("./inputs/topcoder/regs.csv")
CHAG_PATH = Path("./inputs/topcoder/challenge.csv")

In [5]:
db = SeqTopcoder(regs_path=REGS_PATH, chag_path=CHAG_PATH)
db

Read dataset in inputs\topcoder\regs.csv
Original regs shape: (610025, 3)
Original registants size: 60017
Original challenges size: 39916
Filter dataframe shape: (544568, 3)
Index(['challengeId', 'period', 'date', 'prizes', 'technologies', 'platforms'], dtype='object')


In [6]:
db.config_db(batch_size=BATCH,
             shuffle=SHUFFLE,
             num_workers=WORKERS,
             device=DEVICE,
             neg_sample=NEG_SAMPLE)

In [7]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1
INIT_SCALE = 0.001

### Create criterion

In [8]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [9]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x000001DD093D2AF8>, 1, 1)

In [10]:
trans_loss_callback = partial(trans_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_loss_callback

functools.partial(<function trans_loss at 0x000001DD094671F8>, 1, 1, 1)

In [11]:
simple_weight_loss_callback = partial(simple_weight_loss, LINEAR_REG, EMB_REG)
simple_weight_loss_callback

functools.partial(<function simple_weight_loss at 0x000001DD09467288>, 1, 1)

In [12]:
trans_weight_loss_callback = partial(trans_weight_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_weight_loss_callback

functools.partial(<function trans_weight_loss at 0x000001DD09467318>, 1, 1, 1)

### Train model

#### Hyper-parameter

In [13]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1
INIT_SCALE = 0.0001

#### Train FM Model

In [14]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [15]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [33]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [34]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, db)
fm_learner

In [20]:
fm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [29]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [35]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_weight_loss_callback)

In [30]:
fm_learner.fit(epoch=5,
               log_dir=get_log_dir('simple_topcoder', 'fm'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 46546.154640854336
Epoch 0 step 1: training accuarcy: 0.36760000000000004
Epoch 0 step 1: training loss: 44140.504792487875
Epoch 0 step 2: training accuarcy: 0.4319
Epoch 0 step 2: training loss: 41675.07565998279
Epoch 0 step 3: training accuarcy: 0.5108
Epoch 0 step 3: training loss: 39450.99958670329
Epoch 0 step 4: training accuarcy: 0.5817
Epoch 0 step 4: training loss: 37480.27146773438
Epoch 0 step 5: training accuarcy: 0.6623
Epoch 0 step 5: training loss: 35875.01403513422
Epoch 0 step 6: training accuarcy: 0.7083
Epoch 0 step 6: training loss: 34544.81596775044
Epoch 0 step 7: training accuarcy: 0.7407
Epoch 0 step 7: training loss: 33181.54690224211
Epoch 0 step 8: training accuarcy: 0.7913
Epoch 0 step 8: training loss: 32084.20438950817
Epoch 0 step 9: training accuarcy: 0.8089000000000001
Epoch 0 step 9: training loss: 31093.71356452987
Epoch 0 step 10: training accuarcy: 0.8297
Epoch 0 step 10: training loss: 30231.732255741055
Ep

Epoch 0 step 166: training accuarcy: 0.9862000000000001
Epoch 0 step 166: training loss: 2696.7118351185077
Epoch 0 step 167: training accuarcy: 0.9857
Epoch 0 step 167: training loss: 2681.0778855298404
Epoch 0 step 168: training accuarcy: 0.9868
Epoch 0 step 168: training loss: 2626.978286697358
Epoch 0 step 169: training accuarcy: 0.9872000000000001
Epoch 0 step 169: training loss: 2614.225238317762
Epoch 0 step 170: training accuarcy: 0.9849
Epoch 0 step 170: training loss: 2572.3588066466223
Epoch 0 step 171: training accuarcy: 0.9921000000000001
Epoch 0 step 171: training loss: 2562.2526673500697
Epoch 0 step 172: training accuarcy: 0.9870000000000001
Epoch 0 step 172: training loss: 2502.9905367897736
Epoch 0 step 173: training accuarcy: 0.9887
Epoch 0 step 173: training loss: 2456.2448976557566
Epoch 0 step 174: training accuarcy: 0.9894000000000001
Epoch 0 step 174: training loss: 2546.7299974681673
Epoch 0 step 175: training accuarcy: 0.9840000000000001
Epoch 0 step 175: trai

 20%|██████████████████████████████████▍                                                                                                                                         | 1/5 [04:56<19:45, 296.36s/it]

Epoch: 1
Epoch 1 step 263: training loss: 1141.8540969200897
Epoch 1 step 264: training accuarcy: 0.9913000000000001
Epoch 1 step 264: training loss: 1119.3933070159264
Epoch 1 step 265: training accuarcy: 0.995
Epoch 1 step 265: training loss: 1100.372395073889
Epoch 1 step 266: training accuarcy: 0.9957
Epoch 1 step 266: training loss: 1091.5917523604219
Epoch 1 step 267: training accuarcy: 0.995
Epoch 1 step 267: training loss: 1053.180513373251
Epoch 1 step 268: training accuarcy: 0.9955
Epoch 1 step 268: training loss: 1078.1888165328292
Epoch 1 step 269: training accuarcy: 0.9936
Epoch 1 step 269: training loss: 1075.3834084892223
Epoch 1 step 270: training accuarcy: 0.9936
Epoch 1 step 270: training loss: 1027.8584271406962
Epoch 1 step 271: training accuarcy: 0.9966
Epoch 1 step 271: training loss: 1046.4623134771666
Epoch 1 step 272: training accuarcy: 0.9955
Epoch 1 step 272: training loss: 1028.8387499158332
Epoch 1 step 273: training accuarcy: 0.9951000000000001
Epoch 1 ste

...

Epoch: 2
Epoch 2 step 526: training loss: 383.67212274899856
Epoch 2 step 527: training accuarcy: 0.9966
Epoch 2 step 527: training loss: 384.8049289700171
Epoch 2 step 528: training accuarcy: 0.9978
Epoch 2 step 528: training loss: 356.31919884529856
Epoch 2 step 529: training accuarcy: 0.9978
Epoch 2 step 529: training loss: 356.761232820384
Epoch 2 step 530: training accuarcy: 0.999
Epoch 2 step 530: training loss: 362.7373632910302
Epoch 2 step 531: training accuarcy: 0.9981000000000001
Epoch 2 step 531: training loss: 409.8708334306421
Epoch 2 step 532: training accuarcy: 0.9961000000000001
Epoch 2 step 532: training loss: 370.6629892123538
Epoch 2 step 533: training accuarcy: 0.9965
Epoch 2 step 533: training loss: 355.17098244522646
Epoch 2 step 534: training accuarcy: 0.999
Epoch 2 step 534: training loss: 378.93532783609265
Epoch 2 step 535: training accuarcy: 0.9966
Epoch 2 step 535: training loss: 339.88251935452354
Epoch 2 step 536: training accuarcy: 0.9983000000000001
Epo

Epoch 2 step 688: training loss: 375.2483641722556
Epoch 2 step 689: training accuarcy: 0.9983000000000001
Epoch 2 step 689: training loss: 342.9448350660174
Epoch 2 step 690: training accuarcy: 0.9979
Epoch 2 step 690: training loss: 359.4577075171537
Epoch 2 step 691: training accuarcy: 0.9951000000000001
Epoch 2 step 691: training loss: 330.59003584137616
Epoch 2 step 692: training accuarcy: 0.9986
Epoch 2 step 692: training loss: 340.5669023189479
Epoch 2 step 693: training accuarcy: 0.9961000000000001
Epoch 2 step 693: training loss: 350.6996320764523
Epoch 2 step 694: training accuarcy: 0.9964000000000001
Epoch 2 step 694: training loss: 374.09681592900597
Epoch 2 step 695: training accuarcy: 0.9971000000000001
Epoch 2 step 695: training loss: 336.74055270371014
Epoch 2 step 696: training accuarcy: 0.9966
Epoch 2 step 696: training loss: 363.0581063136573
Epoch 2 step 697: training accuarcy: 0.996
Epoch 2 step 697: training loss: 353.8461701288368
Epoch 2 step 698: training accua

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                    | 3/5 [14:28<09:41, 290.78s/it]

Epoch: 3
Epoch 3 step 789: training loss: 296.23032878304855
Epoch 3 step 790: training accuarcy: 0.999
Epoch 3 step 790: training loss: 300.7896643394858
Epoch 3 step 791: training accuarcy: 0.9983000000000001
Epoch 3 step 791: training loss: 293.4835120560487
Epoch 3 step 792: training accuarcy: 0.9989
Epoch 3 step 792: training loss: 285.5181151599255
Epoch 3 step 793: training accuarcy: 0.9991000000000001
Epoch 3 step 793: training loss: 302.478868166378
Epoch 3 step 794: training accuarcy: 0.9982000000000001
Epoch 3 step 794: training loss: 316.27082484827275
Epoch 3 step 795: training accuarcy: 0.9989
Epoch 3 step 795: training loss: 293.5194634860173
Epoch 3 step 796: training accuarcy: 0.9977
Epoch 3 step 796: training loss: 298.44195453152906
Epoch 3 step 797: training accuarcy: 0.9985
Epoch 3 step 797: training loss: 274.5905129792551
Epoch 3 step 798: training accuarcy: 0.9989
Epoch 3 step 798: training loss: 313.0679676110951
Epoch 3 step 799: training accuarcy: 0.9989
Epoc

KeyboardInterrupt: 

In [36]:
fm_learner.fit(epoch=3, 
               log_dir=get_log_dir("weight_topcoder", "fm"))


  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 96099.51260676965
Epoch 0 step 1: training accuarcy: 0.29510000000000003
Epoch 0 step 1: training loss: 88335.88527898806
Epoch 0 step 2: training accuarcy: 0.3618
Epoch 0 step 2: training loss: 82955.90096143921
Epoch 0 step 3: training accuarcy: 0.41050000000000003
Epoch 0 step 3: training loss: 75841.65049929358
Epoch 0 step 4: training accuarcy: 0.47500000000000003
Epoch 0 step 4: training loss: 69900.84115374796
Epoch 0 step 5: training accuarcy: 0.5316000000000001
Epoch 0 step 5: training loss: 66515.16308581155
Epoch 0 step 6: training accuarcy: 0.5718
Epoch 0 step 6: training loss: 62505.37561871747
Epoch 0 step 7: training accuarcy: 0.6056
Epoch 0 step 7: training loss: 57537.948671329985
Epoch 0 step 8: training accuarcy: 0.6546000000000001
Epoch 0 step 8: training loss: 56306.74673244863
Epoch 0 step 9: training accuarcy: 0.6639
Epoch 0 step 9: training loss: 54004.51565162408
Epoch 0 step 10: training accuarcy: 0.6841
Epoch 0 step 10:

Epoch 0 step 166: training accuarcy: 0.9649000000000001
Epoch 0 step 166: training loss: 12025.231994584245
Epoch 0 step 167: training accuarcy: 0.9740000000000001
Epoch 0 step 167: training loss: 12975.623940105927
Epoch 0 step 168: training accuarcy: 0.9597
Epoch 0 step 168: training loss: 12528.237092144336
Epoch 0 step 169: training accuarcy: 0.9664
Epoch 0 step 169: training loss: 12116.927013772427
Epoch 0 step 170: training accuarcy: 0.9694
Epoch 0 step 170: training loss: 12190.54159559381
Epoch 0 step 171: training accuarcy: 0.9705
Epoch 0 step 171: training loss: 11859.6976563459
Epoch 0 step 172: training accuarcy: 0.9723
Epoch 0 step 172: training loss: 12199.927979699658
Epoch 0 step 173: training accuarcy: 0.9687
Epoch 0 step 173: training loss: 12409.036370485526
Epoch 0 step 174: training accuarcy: 0.9647
Epoch 0 step 174: training loss: 11917.692596284536
Epoch 0 step 175: training accuarcy: 0.9752000000000001
Epoch 0 step 175: training loss: 12528.170015238262
Epoch 0


 33%|█████████████████████████████████████████████████████████▎                                                                                                                  | 1/3 [04:50<09:41, 290.77s/it]

Epoch: 1
Epoch 1 step 263: training loss: 8943.806104887635
Epoch 1 step 264: training accuarcy: 0.9852000000000001
Epoch 1 step 264: training loss: 8739.16893308108
Epoch 1 step 265: training accuarcy: 0.9881000000000001
Epoch 1 step 265: training loss: 8939.825616020917
Epoch 1 step 266: training accuarcy: 0.9856
Epoch 1 step 266: training loss: 8735.961095957951
Epoch 1 step 267: training accuarcy: 0.9875
Epoch 1 step 267: training loss: 8881.61574397262
Epoch 1 step 268: training accuarcy: 0.9870000000000001
Epoch 1 step 268: training loss: 8927.158958352806
Epoch 1 step 269: training accuarcy: 0.9848
Epoch 1 step 269: training loss: 8834.69927256897
Epoch 1 step 270: training accuarcy: 0.9830000000000001
Epoch 1 step 270: training loss: 8841.092411649668
Epoch 1 step 271: training accuarcy: 0.9851000000000001
Epoch 1 step 271: training loss: 8631.794868342997
Epoch 1 step 272: training accuarcy: 0.9862000000000001
Epoch 1 step 272: training loss: 8458.694663804956
Epoch 1 step 273

Epoch 1 step 426: training loss: 6221.554330038238
Epoch 1 step 427: training accuarcy: 0.9876
Epoch 1 step 427: training loss: 5927.4489242996915
Epoch 1 step 428: training accuarcy: 0.9934000000000001
Epoch 1 step 428: training loss: 6041.6842210320665
Epoch 1 step 429: training accuarcy: 0.9885
Epoch 1 step 429: training loss: 5944.180503888112
Epoch 1 step 430: training accuarcy: 0.9895
Epoch 1 step 430: training loss: 5981.051606968576
Epoch 1 step 431: training accuarcy: 0.9901000000000001
Epoch 1 step 431: training loss: 5886.426304914507
Epoch 1 step 432: training accuarcy: 0.9914000000000001
Epoch 1 step 432: training loss: 5959.966622372545
Epoch 1 step 433: training accuarcy: 0.9907
Epoch 1 step 433: training loss: 5846.8005121209135
Epoch 1 step 434: training accuarcy: 0.9912000000000001
Epoch 1 step 434: training loss: 5879.018690676422
Epoch 1 step 435: training accuarcy: 0.9896
Epoch 1 step 435: training loss: 5900.992912393967
Epoch 1 step 436: training accuarcy: 0.9900


 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                         | 2/3 [09:35<04:49, 289.03s/it]

Epoch: 2
Epoch 2 step 526: training loss: 4345.961915111372
Epoch 2 step 527: training accuarcy: 0.9978
Epoch 2 step 527: training loss: 4260.850086770885
Epoch 2 step 528: training accuarcy: 0.9973000000000001
Epoch 2 step 528: training loss: 4487.605909708647
Epoch 2 step 529: training accuarcy: 0.9956
Epoch 2 step 529: training loss: 4215.843818764394
Epoch 2 step 530: training accuarcy: 0.9966
Epoch 2 step 530: training loss: 4288.54462929906
Epoch 2 step 531: training accuarcy: 0.9954000000000001
Epoch 2 step 531: training loss: 4238.23354934376
Epoch 2 step 532: training accuarcy: 0.9973000000000001
Epoch 2 step 532: training loss: 4265.6264095508295
Epoch 2 step 533: training accuarcy: 0.997
Epoch 2 step 533: training loss: 4356.499136132832
Epoch 2 step 534: training accuarcy: 0.9961000000000001
Epoch 2 step 534: training loss: 4433.148706856522
Epoch 2 step 535: training accuarcy: 0.9942000000000001
Epoch 2 step 535: training loss: 4427.379311821591
Epoch 2 step 536: training 

Epoch 2 step 689: training accuarcy: 0.9943000000000001
Epoch 2 step 689: training loss: 3416.9060598718124
Epoch 2 step 690: training accuarcy: 0.9939
Epoch 2 step 690: training loss: 3396.018059200225
Epoch 2 step 691: training accuarcy: 0.994
Epoch 2 step 691: training loss: 3282.170893567474
Epoch 2 step 692: training accuarcy: 0.9951000000000001
Epoch 2 step 692: training loss: 3317.985513265985
Epoch 2 step 693: training accuarcy: 0.9928
Epoch 2 step 693: training loss: 3234.8365542682873
Epoch 2 step 694: training accuarcy: 0.9958
Epoch 2 step 694: training loss: 3220.3541611956457
Epoch 2 step 695: training accuarcy: 0.9939
Epoch 2 step 695: training loss: 3209.0935824468315
Epoch 2 step 696: training accuarcy: 0.995
Epoch 2 step 696: training loss: 3167.241672729309
Epoch 2 step 697: training accuarcy: 0.997
Epoch 2 step 697: training loss: 3224.179567806962
Epoch 2 step 698: training accuarcy: 0.9953000000000001
Epoch 2 step 698: training loss: 3485.1692240131806
Epoch 2 step


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [14:23<00:00, 288.59s/it]

In [31]:
del fm_model
T.cuda.empty_cache()

### Train HRM FM Model

In [16]:
hrm_model = TorchHrmFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
hrm_model

TorchHrmFM()

In [17]:
adam_opt = optim.Adam(hrm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [18]:
hrm_learner = FMLearner(hrm_model, adam_opt, schedular, db)
hrm_learner

In [19]:
hrm_learner.compile(train_col='base',
                   valid_col='base',
                   test_col='base',
                   loss_callback=simple_loss_callback)

In [18]:
hrm_learner.compile(train_col='seq',
                    valid_col='seq',
                    test_col='seq',
                    loss_callback=simple_loss_callback)

In [24]:
hrm_learner.compile(train_col='seq',
                    valid_col='seq',
                    test_col='seq',
                    loss_callback=simple_weight_loss_callback)

In [20]:
hrm_learner.fit(epoch=3,
                log_dir=get_log_dir('topcoder', 'hrm'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 42094.005158331034
Epoch 0 step 1: training accuarcy: 0.532
Epoch 0 step 1: training loss: 40873.383274417065
Epoch 0 step 2: training accuarcy: 0.5275
Epoch 0 step 2: training loss: 40043.158675224884
Epoch 0 step 3: training accuarcy: 0.5115000000000001
Epoch 0 step 3: training loss: 38481.62659565021
Epoch 0 step 4: training accuarcy: 0.531
Epoch 0 step 4: training loss: 37545.26783184933
Epoch 0 step 5: training accuarcy: 0.5425
Epoch 0 step 5: training loss: 36458.21108693169
Epoch 0 step 6: training accuarcy: 0.5265
Epoch 0 step 6: training loss: 35192.893540318415
Epoch 0 step 7: training accuarcy: 0.539
Epoch 0 step 7: training loss: 34100.40943591181
Epoch 0 step 8: training accuarcy: 0.5335
Epoch 0 step 8: training loss: 33409.8551280519
Epoch 0 step 9: training accuarcy: 0.522
Epoch 0 step 9: training loss: 32133.455023909646
Epoch 0 step 10: training accuarcy: 0.5275
Epoch 0 step 10: training loss: 31459.402333177084
Epoch 0 step 11: 

Epoch 0 step 172: training accuarcy: 0.724
Epoch 0 step 172: training loss: 1255.015799419085
Epoch 0 step 173: training accuarcy: 0.718
Epoch 0 step 173: training loss: 1243.445423480644
Epoch 0 step 174: training accuarcy: 0.7265
Epoch 0 step 174: training loss: 1241.8855561575454
Epoch 0 step 175: training accuarcy: 0.724
Epoch 0 step 175: training loss: 1240.0318690759455
Epoch 0 step 176: training accuarcy: 0.7165
Epoch 0 step 176: training loss: 1231.1232859525599
Epoch 0 step 177: training accuarcy: 0.726
Epoch 0 step 177: training loss: 1222.4003862039606
Epoch 0 step 178: training accuarcy: 0.7425
Epoch 0 step 178: training loss: 1280.6408639733775
Epoch 0 step 179: training accuarcy: 0.7205
Epoch 0 step 179: training loss: 1266.1483716507191
Epoch 0 step 180: training accuarcy: 0.7155
Epoch 0 step 180: training loss: 1246.5356428763448
Epoch 0 step 181: training accuarcy: 0.7255
Epoch 0 step 181: training loss: 1209.8314671368735
Epoch 0 step 182: training accuarcy: 0.737
Epo

 20%|██████████████████████████████████▍                                                                                                                                         | 1/5 [01:54<07:36, 114.03s/it]

Epoch: 1
Epoch 1 step 263: training loss: 1161.0205454811971
Epoch 1 step 264: training accuarcy: 0.759
Epoch 1 step 264: training loss: 1114.0463723028427
Epoch 1 step 265: training accuarcy: 0.7685
Epoch 1 step 265: training loss: 1137.5079738431443
Epoch 1 step 266: training accuarcy: 0.771
Epoch 1 step 266: training loss: 1146.1399610397661
Epoch 1 step 267: training accuarcy: 0.762
Epoch 1 step 267: training loss: 1140.6525136081366
Epoch 1 step 268: training accuarcy: 0.772
Epoch 1 step 268: training loss: 1142.4505502583102
Epoch 1 step 269: training accuarcy: 0.791
Epoch 1 step 269: training loss: 1133.9076002596264
Epoch 1 step 270: training accuarcy: 0.76
Epoch 1 step 270: training loss: 1134.2826896525069
Epoch 1 step 271: training accuarcy: 0.769
Epoch 1 step 271: training loss: 1135.9700961490314
Epoch 1 step 272: training accuarcy: 0.776
Epoch 1 step 272: training loss: 1128.2721628699794
Epoch 1 step 273: training accuarcy: 0.772
Epoch 1 step 273: training loss: 1142.825

...

Epoch 2 step 697: training loss: 1062.2989079397992
Epoch 2 step 698: training accuarcy: 0.806
Epoch 2 step 698: training loss: 1095.2612995913194
Epoch 2 step 699: training accuarcy: 0.7935
Epoch 2 step 699: training loss: 1065.0421402633988
Epoch 2 step 700: training accuarcy: 0.806
Epoch 2 step 700: training loss: 1039.120796714582
Epoch 2 step 701: training accuarcy: 0.804
Epoch 2 step 701: training loss: 1052.8962720861578
Epoch 2 step 702: training accuarcy: 0.8135
Epoch 2 step 702: training loss: 1053.4870635410366
Epoch 2 step 703: training accuarcy: 0.81
Epoch 2 step 703: training loss: 1048.8249097773485
Epoch 2 step 704: training accuarcy: 0.802
Epoch 2 step 704: training loss: 1060.499785374857
Epoch 2 step 705: training accuarcy: 0.798
Epoch 2 step 705: training loss: 1039.5388385102847
Epoch 2 step 706: training accuarcy: 0.8045
Epoch 2 step 706: training loss: 1079.1362478928888
Epoch 2 step 707: training accuarcy: 0.7995
Epoch 2 step 707: training loss: 1106.80673988984

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                    | 3/5 [05:30<03:42, 111.07s/it]

Epoch: 3
Epoch 3 step 789: training loss: 1066.2613758246357
Epoch 3 step 790: training accuarcy: 0.801
Epoch 3 step 790: training loss: 1074.6224072993346
Epoch 3 step 791: training accuarcy: 0.7935
Epoch 3 step 791: training loss: 1060.1022044013143
Epoch 3 step 792: training accuarcy: 0.8170000000000001
Epoch 3 step 792: training loss: 1034.7502471715661
Epoch 3 step 793: training accuarcy: 0.8170000000000001
Epoch 3 step 793: training loss: 1047.3258581745852
Epoch 3 step 794: training accuarcy: 0.8170000000000001
Epoch 3 step 794: training loss: 1072.122306892444
Epoch 3 step 795: training accuarcy: 0.7915
Epoch 3 step 795: training loss: 1078.9307834525773
Epoch 3 step 796: training accuarcy: 0.799
Epoch 3 step 796: training loss: 1103.5424686332083
Epoch 3 step 797: training accuarcy: 0.777
Epoch 3 step 797: training loss: 1033.3652683804855
Epoch 3 step 798: training accuarcy: 0.8130000000000001
Epoch 3 step 798: training loss: 1071.1659759830459
Epoch 3 step 799: training accu

Epoch 3 step 959: training loss: 1054.0427978074881
Epoch 3 step 960: training accuarcy: 0.795
Epoch 3 step 960: training loss: 1021.0338259692319
Epoch 3 step 961: training accuarcy: 0.799
Epoch 3 step 961: training loss: 1046.5826495096674
Epoch 3 step 962: training accuarcy: 0.811
Epoch 3 step 962: training loss: 1054.2052319673983
Epoch 3 step 963: training accuarcy: 0.8210000000000001
Epoch 3 step 963: training loss: 1122.0936364683673
Epoch 3 step 964: training accuarcy: 0.788
Epoch 3 step 964: training loss: 1052.0740740937154
Epoch 3 step 965: training accuarcy: 0.8055
Epoch 3 step 965: training loss: 1050.083900173126
Epoch 3 step 966: training accuarcy: 0.7985
Epoch 3 step 966: training loss: 1076.7226337107818
Epoch 3 step 967: training accuarcy: 0.794
Epoch 3 step 967: training loss: 1053.1820373913324
Epoch 3 step 968: training accuarcy: 0.8075
Epoch 3 step 968: training loss: 1073.2273617563637
Epoch 3 step 969: training accuarcy: 0.792
Epoch 3 step 969: training loss: 10

KeyboardInterrupt: 

In [25]:
hrm_learner.fit(epoch=3,
                log_dir=get_log_dir('weight_topcoder', 'hrm'))

  0%|                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 193365.16163974453
Epoch 0 step 1: training accuarcy: 0.51
Epoch 0 step 1: training loss: 185119.86205282228
Epoch 0 step 2: training accuarcy: 0.5207
Epoch 0 step 2: training loss: 189420.40994518064
Epoch 0 step 3: training accuarcy: 0.5195000000000001
Epoch 0 step 3: training loss: 179850.18585890112
Epoch 0 step 4: training accuarcy: 0.5253
Epoch 0 step 4: training loss: 176592.3830378908
Epoch 0 step 5: training accuarcy: 0.5118
Epoch 0 step 5: training loss: 171351.80347387
Epoch 0 step 6: training accuarcy: 0.5213
Epoch 0 step 6: training loss: 164100.8087699164
Epoch 0 step 7: training accuarcy: 0.5284
Epoch 0 step 7: training loss: 161553.85251411583
Epoch 0 step 8: training accuarcy: 0.5347000000000001
Epoch 0 step 8: training loss: 162316.5572234422
Epoch 0 step 9: training accuarcy: 0.5323
Epoch 0 step 9: training loss: 153082.98121592737
Epoch 0 step 10: training accuarcy: 0.522
Epoch 0 step 10: training loss: 150206.3848270157
Epoch

Epoch 0 step 169: training loss: 37197.22417312359
Epoch 0 step 170: training accuarcy: 0.6948000000000001
Epoch 0 step 170: training loss: 37271.32799917286
Epoch 0 step 171: training accuarcy: 0.6865
Epoch 0 step 171: training loss: 38013.06343429402
Epoch 0 step 172: training accuarcy: 0.6849000000000001
Epoch 0 step 172: training loss: 37532.25739286332
Epoch 0 step 173: training accuarcy: 0.6824
Epoch 0 step 173: training loss: 37072.33954684183
Epoch 0 step 174: training accuarcy: 0.6904
Epoch 0 step 174: training loss: 36897.478919225105
Epoch 0 step 175: training accuarcy: 0.6941
Epoch 0 step 175: training loss: 36839.429808392735
Epoch 0 step 176: training accuarcy: 0.6974
Epoch 0 step 176: training loss: 36865.56303404251
Epoch 0 step 177: training accuarcy: 0.6892
Epoch 0 step 177: training loss: 37232.450899596624
Epoch 0 step 178: training accuarcy: 0.6881
Epoch 0 step 178: training loss: 36279.362217262074
Epoch 0 step 179: training accuarcy: 0.7096
Epoch 0 step 179: trai

 33%|██████████████▋                             | 1/3 [05:04<10:08, 304.12s/it]

Epoch: 1
Epoch 1 step 263: training loss: 33232.64051768092
Epoch 1 step 264: training accuarcy: 0.7549
Epoch 1 step 264: training loss: 32550.221703703206
Epoch 1 step 265: training accuarcy: 0.7577
Epoch 1 step 265: training loss: 33045.24654042096
Epoch 1 step 266: training accuarcy: 0.756
Epoch 1 step 266: training loss: 32446.43821047878
Epoch 1 step 267: training accuarcy: 0.763
Epoch 1 step 267: training loss: 32815.873558117186
Epoch 1 step 268: training accuarcy: 0.7507
Epoch 1 step 268: training loss: 33085.9551245544
Epoch 1 step 269: training accuarcy: 0.7499
Epoch 1 step 269: training loss: 32525.589556083534
Epoch 1 step 270: training accuarcy: 0.7584000000000001
Epoch 1 step 270: training loss: 33522.12690631198
Epoch 1 step 271: training accuarcy: 0.7422000000000001
Epoch 1 step 271: training loss: 32712.234827701042
Epoch 1 step 272: training accuarcy: 0.7588
Epoch 1 step 272: training loss: 32848.03136463337
Epoch 1 step 273: training accuarcy: 0.7545000000000001
Epoc

Epoch 1 step 429: training accuarcy: 0.7694000000000001
Epoch 1 step 429: training loss: 31576.346707594264
Epoch 1 step 430: training accuarcy: 0.7592
Epoch 1 step 430: training loss: 31232.155290046936
Epoch 1 step 431: training accuarcy: 0.7638
Epoch 1 step 431: training loss: 31054.72281708138
Epoch 1 step 432: training accuarcy: 0.7674000000000001
Epoch 1 step 432: training loss: 30774.843264047104
Epoch 1 step 433: training accuarcy: 0.7667
Epoch 1 step 433: training loss: 30815.03723026877
Epoch 1 step 434: training accuarcy: 0.7647
Epoch 1 step 434: training loss: 30262.529443554227
Epoch 1 step 435: training accuarcy: 0.7756000000000001
Epoch 1 step 435: training loss: 30784.972607255135
Epoch 1 step 436: training accuarcy: 0.769
Epoch 1 step 436: training loss: 31040.70105093181
Epoch 1 step 437: training accuarcy: 0.7639
Epoch 1 step 437: training loss: 30499.637345536426
Epoch 1 step 438: training accuarcy: 0.765
Epoch 1 step 438: training loss: 30841.3660873036
Epoch 1 ste

 67%|█████████████████████████████▎              | 2/3 [10:13<05:05, 305.56s/it]

Epoch: 2
Epoch 2 step 526: training loss: 29444.827389827682
Epoch 2 step 527: training accuarcy: 0.7862
Epoch 2 step 527: training loss: 28523.125714488488
Epoch 2 step 528: training accuarcy: 0.8022
Epoch 2 step 528: training loss: 29318.442319866088
Epoch 2 step 529: training accuarcy: 0.7877000000000001
Epoch 2 step 529: training loss: 29288.732426444403
Epoch 2 step 530: training accuarcy: 0.7866000000000001
Epoch 2 step 530: training loss: 28845.008535668574
Epoch 2 step 531: training accuarcy: 0.7902
Epoch 2 step 531: training loss: 29109.83316338543
Epoch 2 step 532: training accuarcy: 0.7919
Epoch 2 step 532: training loss: 29101.957448656427
Epoch 2 step 533: training accuarcy: 0.79
Epoch 2 step 533: training loss: 29421.874596014342
Epoch 2 step 534: training accuarcy: 0.7853
Epoch 2 step 534: training loss: 29064.878122278165
Epoch 2 step 535: training accuarcy: 0.7874
Epoch 2 step 535: training loss: 29181.4835376725
Epoch 2 step 536: training accuarcy: 0.7942
Epoch 2 step

Epoch 2 step 690: training loss: 28597.40305173381
Epoch 2 step 691: training accuarcy: 0.7861
Epoch 2 step 691: training loss: 28287.801654713963
Epoch 2 step 692: training accuarcy: 0.7956000000000001
Epoch 2 step 692: training loss: 29244.290852748567
Epoch 2 step 693: training accuarcy: 0.7778
Epoch 2 step 693: training loss: 28655.785406814524
Epoch 2 step 694: training accuarcy: 0.7754000000000001
Epoch 2 step 694: training loss: 28746.35303665627
Epoch 2 step 695: training accuarcy: 0.7852
Epoch 2 step 695: training loss: 28817.037135293744
Epoch 2 step 696: training accuarcy: 0.7873
Epoch 2 step 696: training loss: 28885.19211856198
Epoch 2 step 697: training accuarcy: 0.7854
Epoch 2 step 697: training loss: 28467.659738975814
Epoch 2 step 698: training accuarcy: 0.7874
Epoch 2 step 698: training loss: 29266.5728954465
Epoch 2 step 699: training accuarcy: 0.7695000000000001
Epoch 2 step 699: training loss: 28353.769339563405
Epoch 2 step 700: training accuarcy: 0.7932
Epoch 2 s

100%|████████████████████████████████████████████| 3/3 [15:15<00:00, 304.55s/it]


In [21]:
del hrm_model
T.cuda.empty_cache()

### Train PRME FM Model

In [22]:
prme_model = TorchPrmeFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
prme_model

TorchPrmeFM()

In [23]:
adam_opt = optim.Adam(prme_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [24]:
prme_learner = FMLearner(prme_model, adam_opt, schedular, db)
prme_learner

In [25]:
prme_learner.compile(train_col='base',
                     valid_col='base',
                     test_col='base',
                     loss_callback=simple_loss_callback)

In [34]:
prme_learner.compile(train_col='seq',
                     valid_col='seq',
                     test_col='seq',
                     loss_callback=simple_loss_callback)

In [40]:
prme_learner.compile(train_col='seq',
                     valid_col='seq',
                     test_col='seq',
                     loss_callback=simple_weight_loss_callback)

In [26]:
prme_learner.fit(epoch=3,
                 log_dir=get_log_dir('topcoder', 'prme'))


  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 45930.1427652565
Epoch 0 step 1: training accuarcy: 0.529
Epoch 0 step 1: training loss: 44926.31891158488
Epoch 0 step 2: training accuarcy: 0.529
Epoch 0 step 2: training loss: 43107.71907088262
Epoch 0 step 3: training accuarcy: 0.546
Epoch 0 step 3: training loss: 42306.41889813116
Epoch 0 step 4: training accuarcy: 0.5315
Epoch 0 step 4: training loss: 41712.817676772436
Epoch 0 step 5: training accuarcy: 0.5095000000000001
Epoch 0 step 5: training loss: 40558.056182405955
Epoch 0 step 6: training accuarcy: 0.53
Epoch 0 step 6: training loss: 38942.33300396817
Epoch 0 step 7: training accuarcy: 0.539
Epoch 0 step 7: training loss: 38302.398712509435
Epoch 0 step 8: training accuarcy: 0.5415
Epoch 0 step 8: training loss: 37441.500794490676
Epoch 0 step 9: training accuarcy: 0.543
Epoch 0 step 9: training loss: 36332.164361024115
Epoch 0 step 10: training accuarcy: 0.54
Epoch 0 step 10: training loss: 34859.21795184726
Epoch 0 step 11: traini

Epoch 0 step 172: training accuarcy: 0.72
Epoch 0 step 172: training loss: 1442.1492595142863
Epoch 0 step 173: training accuarcy: 0.722
Epoch 0 step 173: training loss: 1384.6518471886536
Epoch 0 step 174: training accuarcy: 0.755
Epoch 0 step 174: training loss: 1422.8648673078271
Epoch 0 step 175: training accuarcy: 0.733
Epoch 0 step 175: training loss: 1400.6681082110008
Epoch 0 step 176: training accuarcy: 0.7335
Epoch 0 step 176: training loss: 1423.3400651094141
Epoch 0 step 177: training accuarcy: 0.7245
Epoch 0 step 177: training loss: 1384.8729873377806
Epoch 0 step 178: training accuarcy: 0.7275
Epoch 0 step 178: training loss: 1398.7661499754358
Epoch 0 step 179: training accuarcy: 0.717
Epoch 0 step 179: training loss: 1391.0536320455753
Epoch 0 step 180: training accuarcy: 0.7105
Epoch 0 step 180: training loss: 1385.547574740684
Epoch 0 step 181: training accuarcy: 0.7355
Epoch 0 step 181: training loss: 1382.0024146235376
Epoch 0 step 182: training accuarcy: 0.7185
Epo


 33%|█████████████████████████████████████████████████████████▎                                                                                                                  | 1/3 [01:50<03:41, 110.61s/it]

Epoch: 1
Epoch 1 step 263: training loss: 1167.1539860411112
Epoch 1 step 264: training accuarcy: 0.76
Epoch 1 step 264: training loss: 1181.8496994763996
Epoch 1 step 265: training accuarcy: 0.776
Epoch 1 step 265: training loss: 1159.0912795804231
Epoch 1 step 266: training accuarcy: 0.7625000000000001
Epoch 1 step 266: training loss: 1169.263601847875
Epoch 1 step 267: training accuarcy: 0.776
Epoch 1 step 267: training loss: 1138.7867774106874
Epoch 1 step 268: training accuarcy: 0.77
Epoch 1 step 268: training loss: 1159.3624794780665
Epoch 1 step 269: training accuarcy: 0.7725
Epoch 1 step 269: training loss: 1220.2681861367
Epoch 1 step 270: training accuarcy: 0.752
Epoch 1 step 270: training loss: 1158.408705790937
Epoch 1 step 271: training accuarcy: 0.801
Epoch 1 step 271: training loss: 1165.451715004065
Epoch 1 step 272: training accuarcy: 0.766
Epoch 1 step 272: training loss: 1160.3018765540446
Epoch 1 step 273: training accuarcy: 0.765
Epoch 1 step 273: training loss: 11

Epoch 1 step 433: training loss: 1116.6090469699702
Epoch 1 step 434: training accuarcy: 0.7685
Epoch 1 step 434: training loss: 1109.9365692266792
Epoch 1 step 435: training accuarcy: 0.7815
Epoch 1 step 435: training loss: 1091.6676773468048
Epoch 1 step 436: training accuarcy: 0.8035
Epoch 1 step 436: training loss: 1094.0061469619645
Epoch 1 step 437: training accuarcy: 0.783
Epoch 1 step 437: training loss: 1090.9326863794192
Epoch 1 step 438: training accuarcy: 0.804
Epoch 1 step 438: training loss: 1097.8986508429678
Epoch 1 step 439: training accuarcy: 0.7845
Epoch 1 step 439: training loss: 1082.479106029879
Epoch 1 step 440: training accuarcy: 0.789
Epoch 1 step 440: training loss: 1077.5749124581898
Epoch 1 step 441: training accuarcy: 0.805
Epoch 1 step 441: training loss: 1117.718832860423
Epoch 1 step 442: training accuarcy: 0.7855
Epoch 1 step 442: training loss: 1072.5240612646796
Epoch 1 step 443: training accuarcy: 0.7965
Epoch 1 step 443: training loss: 1104.07914481


 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                         | 2/3 [03:43<01:51, 111.22s/it]

Epoch: 2
Epoch 2 step 526: training loss: 1079.2565521023134
Epoch 2 step 527: training accuarcy: 0.8180000000000001
Epoch 2 step 527: training loss: 1054.6608990216007
Epoch 2 step 528: training accuarcy: 0.81
Epoch 2 step 528: training loss: 1081.114045001487
Epoch 2 step 529: training accuarcy: 0.8140000000000001
Epoch 2 step 529: training loss: 1129.2531142538235
Epoch 2 step 530: training accuarcy: 0.785
Epoch 2 step 530: training loss: 1075.2504198332804
Epoch 2 step 531: training accuarcy: 0.803
Epoch 2 step 531: training loss: 1097.8783071993746
Epoch 2 step 532: training accuarcy: 0.8
Epoch 2 step 532: training loss: 1124.0145303906029
Epoch 2 step 533: training accuarcy: 0.8005
Epoch 2 step 533: training loss: 1085.8969670247088
Epoch 2 step 534: training accuarcy: 0.809
Epoch 2 step 534: training loss: 1069.1565002380053
Epoch 2 step 535: training accuarcy: 0.798
Epoch 2 step 535: training loss: 1105.3322161568171
Epoch 2 step 536: training accuarcy: 0.7855
Epoch 2 step 536:

Epoch 2 step 696: training accuarcy: 0.805
Epoch 2 step 696: training loss: 1088.4502864821359
Epoch 2 step 697: training accuarcy: 0.81
Epoch 2 step 697: training loss: 1047.7908373618059
Epoch 2 step 698: training accuarcy: 0.808
Epoch 2 step 698: training loss: 1097.6841731657648
Epoch 2 step 699: training accuarcy: 0.804
Epoch 2 step 699: training loss: 1032.5952040628438
Epoch 2 step 700: training accuarcy: 0.8145
Epoch 2 step 700: training loss: 1020.6399961645035
Epoch 2 step 701: training accuarcy: 0.8210000000000001
Epoch 2 step 701: training loss: 1094.6456881295787
Epoch 2 step 702: training accuarcy: 0.791
Epoch 2 step 702: training loss: 1050.0602117666988
Epoch 2 step 703: training accuarcy: 0.8115
Epoch 2 step 703: training loss: 1031.924193885374
Epoch 2 step 704: training accuarcy: 0.811
Epoch 2 step 704: training loss: 1041.7253006272836
Epoch 2 step 705: training accuarcy: 0.8165
Epoch 2 step 705: training loss: 1049.305623754387
Epoch 2 step 706: training accuarcy: 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [05:33<00:00, 111.19s/it]


In [41]:
prme_learner.fit(epoch=3,
                 log_dir=get_log_dir('weight_topcoder', 'prme'))

  0%|                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 294606.13853030634
Epoch 0 step 1: training accuarcy: 0.5142
Epoch 0 step 1: training loss: 286108.444642681
Epoch 0 step 2: training accuarcy: 0.5282
Epoch 0 step 2: training loss: 293987.4134806083
Epoch 0 step 3: training accuarcy: 0.5083
Epoch 0 step 3: training loss: 284043.4721217331
Epoch 0 step 4: training accuarcy: 0.5192
Epoch 0 step 4: training loss: 278149.06834666454
Epoch 0 step 5: training accuarcy: 0.5146000000000001
Epoch 0 step 5: training loss: 279162.83377816185
Epoch 0 step 6: training accuarcy: 0.5137
Epoch 0 step 6: training loss: 263925.41081593785
Epoch 0 step 7: training accuarcy: 0.533
Epoch 0 step 7: training loss: 272980.8215130734
Epoch 0 step 8: training accuarcy: 0.5086
Epoch 0 step 8: training loss: 261422.55033331588
Epoch 0 step 9: training accuarcy: 0.5252
Epoch 0 step 9: training loss: 246522.51367903984
Epoch 0 step 10: training accuarcy: 0.5244
Epoch 0 step 10: training loss: 247796.70586116522
Epoch 0 step 

Epoch 0 step 86: training loss: 49351.753423952265
Epoch 0 step 87: training accuarcy: 0.5918
Epoch 0 step 87: training loss: 50024.61976459105
Epoch 0 step 88: training accuarcy: 0.5839
Epoch 0 step 88: training loss: 49202.58714893614
Epoch 0 step 89: training accuarcy: 0.5897
Epoch 0 step 89: training loss: 48909.41989555757
Epoch 0 step 90: training accuarcy: 0.5923
Epoch 0 step 90: training loss: 48638.808378756796
Epoch 0 step 91: training accuarcy: 0.5889
Epoch 0 step 91: training loss: 49006.547890239744
Epoch 0 step 92: training accuarcy: 0.5831000000000001
Epoch 0 step 92: training loss: 47676.21878923138
Epoch 0 step 93: training accuarcy: 0.6065
Epoch 0 step 93: training loss: 48449.83790047556
Epoch 0 step 94: training accuarcy: 0.5983
Epoch 0 step 94: training loss: 47852.71035297997
Epoch 0 step 95: training accuarcy: 0.6043000000000001
Epoch 0 step 95: training loss: 47793.16903701683
Epoch 0 step 96: training accuarcy: 0.6057
Epoch 0 step 96: training loss: 47773.42434

Epoch 0 step 253: training loss: 35283.5836472917
Epoch 0 step 254: training accuarcy: 0.7287
Epoch 0 step 254: training loss: 36261.26602595637
Epoch 0 step 255: training accuarcy: 0.7130000000000001
Epoch 0 step 255: training loss: 35349.418825336514
Epoch 0 step 256: training accuarcy: 0.7243
Epoch 0 step 256: training loss: 35795.27394716085
Epoch 0 step 257: training accuarcy: 0.7185
Epoch 0 step 257: training loss: 35761.73537550089
Epoch 0 step 258: training accuarcy: 0.7175
Epoch 0 step 258: training loss: 35705.99420170844
Epoch 0 step 259: training accuarcy: 0.7271000000000001
Epoch 0 step 259: training loss: 35823.21189438501
Epoch 0 step 260: training accuarcy: 0.7209
Epoch 0 step 260: training loss: 36391.402454865245
Epoch 0 step 261: training accuarcy: 0.7184
Epoch 0 step 261: training loss: 35469.310334724505
Epoch 0 step 262: training accuarcy: 0.7233
Epoch 0 step 262: training loss: 19016.696263744274
Epoch 0 step 263: training accuarcy: 0.7282051282051282
Epoch 0: tr

 33%|██████████████▋                             | 1/3 [04:44<09:28, 284.08s/it]

...

Epoch 1 step 428: training accuarcy: 0.7621
Epoch 1 step 428: training loss: 31340.68653146692
Epoch 1 step 429: training accuarcy: 0.7647
Epoch 1 step 429: training loss: 31231.03471677229
Epoch 1 step 430: training accuarcy: 0.7698
Epoch 1 step 430: training loss: 31528.196175496978
Epoch 1 step 431: training accuarcy: 0.7651
Epoch 1 step 431: training loss: 31313.9731503945
Epoch 1 step 432: training accuarcy: 0.7635000000000001
Epoch 1 step 432: training loss: 32011.114723363236
Epoch 1 step 433: training accuarcy: 0.7627
Epoch 1 step 433: training loss: 31685.05147831228
Epoch 1 step 434: training accuarcy: 0.7606
Epoch 1 step 434: training loss: 31442.05208022455
Epoch 1 step 435: training accuarcy: 0.7678
Epoch 1 step 435: training loss: 30933.32182664942
Epoch 1 step 436: training accuarcy: 0.7799
Epoch 1 step 436: training loss: 31461.20992526184
Epoch 1 step 437: training accuarcy: 0.769
Epoch 1 step 437: training loss: 31561.013876726684
Epoch 1 step 438: training accuarcy: 

 67%|█████████████████████████████▎              | 2/3 [09:40<04:47, 287.67s/it]

Epoch: 2
Epoch 2 step 526: training loss: 29592.70226770191
Epoch 2 step 527: training accuarcy: 0.7885000000000001
Epoch 2 step 527: training loss: 29542.998646134674
Epoch 2 step 528: training accuarcy: 0.7934
Epoch 2 step 528: training loss: 29294.765387400574
Epoch 2 step 529: training accuarcy: 0.7911
Epoch 2 step 529: training loss: 29082.70863222242
Epoch 2 step 530: training accuarcy: 0.7921
Epoch 2 step 530: training loss: 29819.621505648596
Epoch 2 step 531: training accuarcy: 0.7924
Epoch 2 step 531: training loss: 30067.853013730157
Epoch 2 step 532: training accuarcy: 0.781
Epoch 2 step 532: training loss: 29661.87719487754
Epoch 2 step 533: training accuarcy: 0.7889
Epoch 2 step 533: training loss: 29798.27668340026
Epoch 2 step 534: training accuarcy: 0.79
Epoch 2 step 534: training loss: 29361.29082672926
Epoch 2 step 535: training accuarcy: 0.7918000000000001
Epoch 2 step 535: training loss: 29210.004528644535
Epoch 2 step 536: training accuarcy: 0.8016000000000001
Epo

Epoch 2 step 690: training loss: 28686.99754215468
Epoch 2 step 691: training accuarcy: 0.7906000000000001
Epoch 2 step 691: training loss: 28763.448810794554
Epoch 2 step 692: training accuarcy: 0.7953
Epoch 2 step 692: training loss: 28734.754986791093
Epoch 2 step 693: training accuarcy: 0.7903
Epoch 2 step 693: training loss: 28832.191333561503
Epoch 2 step 694: training accuarcy: 0.7874
Epoch 2 step 694: training loss: 28889.438832097236
Epoch 2 step 695: training accuarcy: 0.7891
Epoch 2 step 695: training loss: 29524.77223648897
Epoch 2 step 696: training accuarcy: 0.7811
Epoch 2 step 696: training loss: 28111.733619486877
Epoch 2 step 697: training accuarcy: 0.793
Epoch 2 step 697: training loss: 28398.059954069533
Epoch 2 step 698: training accuarcy: 0.7891
Epoch 2 step 698: training loss: 28536.42865893506
Epoch 2 step 699: training accuarcy: 0.7935
Epoch 2 step 699: training loss: 28458.421778173768
Epoch 2 step 700: training accuarcy: 0.793
Epoch 2 step 700: training loss: 

100%|████████████████████████████████████████████| 3/3 [14:33<00:00, 289.31s/it]


In [27]:
del prme_model
T.cuda.empty_cache()

### Train Trans FM Model

In [28]:
trans_model = TorchTransFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
trans_model

TorchTransFM()

In [29]:
adam_opt = optim.Adam(trans_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [30]:
trans_learner = FMLearner(trans_model, adam_opt, schedular, db)
trans_learner

In [31]:
trans_learner.compile(train_col='base',
                      valid_col='base',
                      test_col='base',
                      loss_callback=trans_loss_callback)

In [46]:
trans_learner.compile(train_col='seq',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=trans_loss_callback)

In [52]:
trans_learner.compile(train_col='seq',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=trans_weight_loss_callback)

In [32]:
trans_learner.fit(epoch=3,
                  log_dir=get_log_dir('topcoder', 'trans'))


  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 46964.64759320144
Epoch 0 step 1: training accuarcy: 0.5305
Epoch 0 step 1: training loss: 46303.860487355196
Epoch 0 step 2: training accuarcy: 0.52
Epoch 0 step 2: training loss: 44911.01438392426
Epoch 0 step 3: training accuarcy: 0.5285
Epoch 0 step 3: training loss: 43899.87006394936
Epoch 0 step 4: training accuarcy: 0.5215
Epoch 0 step 4: training loss: 42597.08275518355
Epoch 0 step 5: training accuarcy: 0.532
Epoch 0 step 5: training loss: 41179.186404459506
Epoch 0 step 6: training accuarcy: 0.5465
Epoch 0 step 6: training loss: 40095.95313489541
Epoch 0 step 7: training accuarcy: 0.5305
Epoch 0 step 7: training loss: 39139.03911682185
Epoch 0 step 8: training accuarcy: 0.5325
Epoch 0 step 8: training loss: 38596.50010456356
Epoch 0 step 9: training accuarcy: 0.5225
Epoch 0 step 9: training loss: 36910.07042390128
Epoch 0 step 10: training accuarcy: 0.5495
Epoch 0 step 10: training loss: 36128.559668267524
Epoch 0 step 11: training accu

Epoch 0 step 172: training accuarcy: 0.8305
Epoch 0 step 172: training loss: 992.0807289740117
Epoch 0 step 173: training accuarcy: 0.8315
Epoch 0 step 173: training loss: 968.7280818279737
Epoch 0 step 174: training accuarcy: 0.8365
Epoch 0 step 174: training loss: 961.1830040256607
Epoch 0 step 175: training accuarcy: 0.8425
Epoch 0 step 175: training loss: 951.0140695935105
Epoch 0 step 176: training accuarcy: 0.8445
Epoch 0 step 176: training loss: 982.992963441875
Epoch 0 step 177: training accuarcy: 0.833
Epoch 0 step 177: training loss: 954.1620598194033
Epoch 0 step 178: training accuarcy: 0.841
Epoch 0 step 178: training loss: 921.3220892434172
Epoch 0 step 179: training accuarcy: 0.8415
Epoch 0 step 179: training loss: 940.4602866947916
Epoch 0 step 180: training accuarcy: 0.841
Epoch 0 step 180: training loss: 936.1212626623777
Epoch 0 step 181: training accuarcy: 0.846
Epoch 0 step 181: training loss: 905.4779193482661
Epoch 0 step 182: training accuarcy: 0.8415
Epoch 0 ste


 33%|█████████████████████████████████████████████████████████▎                                                                                                                  | 1/3 [02:00<04:00, 120.34s/it]

Epoch: 1
Epoch 1 step 263: training loss: 701.8108700745743
Epoch 1 step 264: training accuarcy: 0.8915000000000001
Epoch 1 step 264: training loss: 677.0688283020372
Epoch 1 step 265: training accuarcy: 0.899
Epoch 1 step 265: training loss: 697.5448142336828
Epoch 1 step 266: training accuarcy: 0.8875000000000001
Epoch 1 step 266: training loss: 736.2438204047985
Epoch 1 step 267: training accuarcy: 0.8795000000000001
Epoch 1 step 267: training loss: 728.9153548235302
Epoch 1 step 268: training accuarcy: 0.8785000000000001
Epoch 1 step 268: training loss: 679.923000152653
Epoch 1 step 269: training accuarcy: 0.901
Epoch 1 step 269: training loss: 680.8993721859106
Epoch 1 step 270: training accuarcy: 0.8925000000000001
Epoch 1 step 270: training loss: 716.9525343916167
Epoch 1 step 271: training accuarcy: 0.8855000000000001
Epoch 1 step 271: training loss: 692.0368629753517
Epoch 1 step 272: training accuarcy: 0.8915000000000001
Epoch 1 step 272: training loss: 703.0588028288532
Epoc

Epoch 1 step 433: training loss: 618.0906297669151
Epoch 1 step 434: training accuarcy: 0.906
Epoch 1 step 434: training loss: 629.1722150466322
Epoch 1 step 435: training accuarcy: 0.9035
Epoch 1 step 435: training loss: 593.9876006586827
Epoch 1 step 436: training accuarcy: 0.916
Epoch 1 step 436: training loss: 651.4552892530296
Epoch 1 step 437: training accuarcy: 0.905
Epoch 1 step 437: training loss: 580.8252509945722
Epoch 1 step 438: training accuarcy: 0.916
Epoch 1 step 438: training loss: 607.671725267024
Epoch 1 step 439: training accuarcy: 0.907
Epoch 1 step 439: training loss: 619.405723904622
Epoch 1 step 440: training accuarcy: 0.9055
Epoch 1 step 440: training loss: 620.0468855746135
Epoch 1 step 441: training accuarcy: 0.9065
Epoch 1 step 441: training loss: 620.5297027854897
Epoch 1 step 442: training accuarcy: 0.9015
Epoch 1 step 442: training loss: 613.918209907714
Epoch 1 step 443: training accuarcy: 0.9055
Epoch 1 step 443: training loss: 654.6236817969592
Epoch 1


 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                         | 2/3 [03:59<01:59, 119.97s/it]

Epoch: 2
Epoch 2 step 526: training loss: 552.0526878855613
Epoch 2 step 527: training accuarcy: 0.9185
Epoch 2 step 527: training loss: 534.5683291432863
Epoch 2 step 528: training accuarcy: 0.924
Epoch 2 step 528: training loss: 548.1074612817788
Epoch 2 step 529: training accuarcy: 0.927
Epoch 2 step 529: training loss: 559.8465337082757
Epoch 2 step 530: training accuarcy: 0.9235
Epoch 2 step 530: training loss: 531.1384140414998
Epoch 2 step 531: training accuarcy: 0.9305
Epoch 2 step 531: training loss: 558.5680281442114
Epoch 2 step 532: training accuarcy: 0.916
Epoch 2 step 532: training loss: 542.7798058085571
Epoch 2 step 533: training accuarcy: 0.9265
Epoch 2 step 533: training loss: 548.968407527061
Epoch 2 step 534: training accuarcy: 0.9285
Epoch 2 step 534: training loss: 522.6574790028385
Epoch 2 step 535: training accuarcy: 0.925
Epoch 2 step 535: training loss: 552.5348446817952
Epoch 2 step 536: training accuarcy: 0.9295
Epoch 2 step 536: training loss: 549.165184995

Epoch 2 step 699: training loss: 546.2942293102979
Epoch 2 step 700: training accuarcy: 0.9215
Epoch 2 step 700: training loss: 557.2768137697982
Epoch 2 step 701: training accuarcy: 0.919
Epoch 2 step 701: training loss: 542.0525564088518
Epoch 2 step 702: training accuarcy: 0.925
Epoch 2 step 702: training loss: 582.9115506085553
Epoch 2 step 703: training accuarcy: 0.9185
Epoch 2 step 703: training loss: 518.5597576352862
Epoch 2 step 704: training accuarcy: 0.9205
Epoch 2 step 704: training loss: 539.4045196849677
Epoch 2 step 705: training accuarcy: 0.916
Epoch 2 step 705: training loss: 515.2406695714028
Epoch 2 step 706: training accuarcy: 0.934
Epoch 2 step 706: training loss: 533.4209577365324
Epoch 2 step 707: training accuarcy: 0.932
Epoch 2 step 707: training loss: 544.621908566643
Epoch 2 step 708: training accuarcy: 0.9215
Epoch 2 step 708: training loss: 507.03076866304343
Epoch 2 step 709: training accuarcy: 0.9385
Epoch 2 step 709: training loss: 499.1550378772791
Epoc


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [06:04<00:00, 121.49s/it]


In [53]:
trans_learner.fit(epoch=3,
                  log_dir=get_log_dir('weight_topcoder', 'trans'))

  0%|                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 328693.0835166554
Epoch 0 step 1: training accuarcy: 0.4978
Epoch 0 step 1: training loss: 314058.7249141159
Epoch 0 step 2: training accuarcy: 0.5168
Epoch 0 step 2: training loss: 300176.1108415055
Epoch 0 step 3: training accuarcy: 0.5159
Epoch 0 step 3: training loss: 307014.09675405966
Epoch 0 step 4: training accuarcy: 0.5114000000000001
Epoch 0 step 4: training loss: 297557.54419490095
Epoch 0 step 5: training accuarcy: 0.5051
Epoch 0 step 5: training loss: 291071.6806946936
Epoch 0 step 6: training accuarcy: 0.5202
Epoch 0 step 6: training loss: 281828.43845424784
Epoch 0 step 7: training accuarcy: 0.5189
Epoch 0 step 7: training loss: 268520.46315127116
Epoch 0 step 8: training accuarcy: 0.5279
Epoch 0 step 8: training loss: 272973.084331861
Epoch 0 step 9: training accuarcy: 0.5239
Epoch 0 step 9: training loss: 262233.4747713839
Epoch 0 step 10: training accuarcy: 0.5155000000000001
Epoch 0 step 10: training loss: 253659.75581515126
Ep

Epoch 0 step 166: training loss: 26601.093323936373
Epoch 0 step 167: training accuarcy: 0.8469
Epoch 0 step 167: training loss: 26099.55135480284
Epoch 0 step 168: training accuarcy: 0.8402000000000001
Epoch 0 step 168: training loss: 25822.06086071969
Epoch 0 step 169: training accuarcy: 0.8493
Epoch 0 step 169: training loss: 26762.53642552209
Epoch 0 step 170: training accuarcy: 0.8391000000000001
Epoch 0 step 170: training loss: 25803.678595341225
Epoch 0 step 171: training accuarcy: 0.8464
Epoch 0 step 171: training loss: 25582.729381492667
Epoch 0 step 172: training accuarcy: 0.8496
Epoch 0 step 172: training loss: 26239.21400536701
Epoch 0 step 173: training accuarcy: 0.8427
Epoch 0 step 173: training loss: 25087.77036797405
Epoch 0 step 174: training accuarcy: 0.8572000000000001
Epoch 0 step 174: training loss: 25512.108946995184
Epoch 0 step 175: training accuarcy: 0.8507
Epoch 0 step 175: training loss: 25481.058037329185
Epoch 0 step 176: training accuarcy: 0.85030000000000

 33%|██████████████▋                             | 1/3 [05:16<10:32, 316.16s/it]

Epoch: 1
Epoch 1 step 263: training loss: 18654.763376803552
Epoch 1 step 264: training accuarcy: 0.904
Epoch 1 step 264: training loss: 19169.8509397557
Epoch 1 step 265: training accuarcy: 0.8995000000000001
Epoch 1 step 265: training loss: 18744.404241145487
Epoch 1 step 266: training accuarcy: 0.9029
Epoch 1 step 266: training loss: 19100.47757906397
Epoch 1 step 267: training accuarcy: 0.8979
Epoch 1 step 267: training loss: 19022.88913226355
Epoch 1 step 268: training accuarcy: 0.8978
Epoch 1 step 268: training loss: 18556.884732716415
Epoch 1 step 269: training accuarcy: 0.9047000000000001
Epoch 1 step 269: training loss: 18892.98198253762
Epoch 1 step 270: training accuarcy: 0.9004000000000001
Epoch 1 step 270: training loss: 18943.752710191136
Epoch 1 step 271: training accuarcy: 0.8983000000000001
Epoch 1 step 271: training loss: 18532.35758176867
Epoch 1 step 272: training accuarcy: 0.8993
Epoch 1 step 272: training loss: 18644.70328773265
Epoch 1 step 273: training accuarcy

Epoch 1 step 427: training loss: 15603.772535963177
Epoch 1 step 428: training accuarcy: 0.9072
Epoch 1 step 428: training loss: 15564.097606553598
Epoch 1 step 429: training accuarcy: 0.902
Epoch 1 step 429: training loss: 15377.576241857663
Epoch 1 step 430: training accuarcy: 0.9052
Epoch 1 step 430: training loss: 15162.334503598253
Epoch 1 step 431: training accuarcy: 0.9084000000000001
Epoch 1 step 431: training loss: 15015.394849332051
Epoch 1 step 432: training accuarcy: 0.9085000000000001
Epoch 1 step 432: training loss: 15004.000818654247
Epoch 1 step 433: training accuarcy: 0.9114000000000001
Epoch 1 step 433: training loss: 15325.914791305458
Epoch 1 step 434: training accuarcy: 0.9122
Epoch 1 step 434: training loss: 14814.798455274773
Epoch 1 step 435: training accuarcy: 0.9128000000000001
Epoch 1 step 435: training loss: 15250.127153931393
Epoch 1 step 436: training accuarcy: 0.9052
Epoch 1 step 436: training loss: 14903.909304156427
Epoch 1 step 437: training accuarcy: 

 67%|█████████████████████████████▎              | 2/3 [10:19<05:12, 312.32s/it]

Epoch: 2
Epoch 2 step 526: training loss: 12579.079971786003
Epoch 2 step 527: training accuarcy: 0.932
Epoch 2 step 527: training loss: 12467.619066556137
Epoch 2 step 528: training accuarcy: 0.9273
Epoch 2 step 528: training loss: 12956.344311152803
Epoch 2 step 529: training accuarcy: 0.9261
Epoch 2 step 529: training loss: 12257.875309358122
Epoch 2 step 530: training accuarcy: 0.9339000000000001
Epoch 2 step 530: training loss: 12327.81546683406
Epoch 2 step 531: training accuarcy: 0.934
Epoch 2 step 531: training loss: 12897.246040983051
Epoch 2 step 532: training accuarcy: 0.9271
Epoch 2 step 532: training loss: 12555.067693549696
Epoch 2 step 533: training accuarcy: 0.9292
Epoch 2 step 533: training loss: 12355.227634627197
Epoch 2 step 534: training accuarcy: 0.9334
Epoch 2 step 534: training loss: 12954.758991505158
Epoch 2 step 535: training accuarcy: 0.9284
Epoch 2 step 535: training loss: 12456.148345488487
Epoch 2 step 536: training accuarcy: 0.9321
Epoch 2 step 536: trai

Epoch 2 step 690: training accuarcy: 0.9318000000000001
Epoch 2 step 690: training loss: 11941.249305893562
Epoch 2 step 691: training accuarcy: 0.9281
Epoch 2 step 691: training loss: 11631.430190287456
Epoch 2 step 692: training accuarcy: 0.9298000000000001
Epoch 2 step 692: training loss: 11589.049984246412
Epoch 2 step 693: training accuarcy: 0.9275
Epoch 2 step 693: training loss: 11455.207522423885
Epoch 2 step 694: training accuarcy: 0.9323
Epoch 2 step 694: training loss: 11613.57102550763
Epoch 2 step 695: training accuarcy: 0.9311
Epoch 2 step 695: training loss: 11602.339899947161
Epoch 2 step 696: training accuarcy: 0.9281
Epoch 2 step 696: training loss: 11640.08287377311
Epoch 2 step 697: training accuarcy: 0.9309000000000001
Epoch 2 step 697: training loss: 11676.68072996902
Epoch 2 step 698: training accuarcy: 0.9277000000000001
Epoch 2 step 698: training loss: 11423.882087064068
Epoch 2 step 699: training accuarcy: 0.9314
Epoch 2 step 699: training loss: 11673.40305939

100%|████████████████████████████████████████████| 3/3 [15:24<00:00, 310.22s/it]


In [33]:
del trans_model
T.cuda.empty_cache()

### Train Trans Probability Model

In [15]:
prob_model = TransProbFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN, init_scale=INIT_SCALE)
prob_model

TransProbFM()

In [16]:
adam_opt = optim.Adam(prob_model.parameters(), lr=0.01)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=100,
                                      gamma=0.5)

In [17]:
prob_learner = FMLearner(prob_model, adam_opt, schedular, db)
prob_learner

In [18]:
prob_learner.compile(train_col='base',
                     valid_col='base',
                     test_col='base',
                     loss_callback=trans_loss_callback)

In [ ]:
prob_learner.fit(epoch=4, log_dir=get_log_dir('topcoder', 'prob'))

  0%|                                                                                                                                                           | 0/4 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 46914.52543605391
Epoch 0 step 1: training accuarcy: 0.5265
Epoch 0 step 1: training loss: 35919.0044689248
Epoch 0 step 2: training accuarcy: 0.5395
Epoch 0 step 2: training loss: 27242.175542358495
Epoch 0 step 3: training accuarcy: 0.523
Epoch 0 step 3: training loss: 19556.374216549906
Epoch 0 step 4: training accuarcy: 0.5345
Epoch 0 step 4: training loss: 14213.189092008139
Epoch 0 step 5: training accuarcy: 0.5625
Epoch 0 step 5: training loss: 10640.469354668963
Epoch 0 step 6: training accuarcy: 0.555
Epoch 0 step 6: training loss: 8382.043762964946
Epoch 0 step 7: training accuarcy: 0.557
Epoch 0 step 7: training loss: 7225.966130737879
Epoch 0 step 8: training accuarcy: 0.5535
Epoch 0 step 8: training loss: 6239.35087284336
Epoch 0 step 9: training accuarcy: 0.5795
Epoch 0 step 9: training loss: 5779.441470311422
Epoch 0 step 10: training accuarcy: 0.578
Epoch 0 step 10: training loss: 5959.56671379762
Epoch 0 step 11: training accuarc

Epoch 0 step 87: training accuarcy: 0.8180000000000001
Epoch 0 step 87: training loss: 4308.806507685686
Epoch 0 step 88: training accuarcy: 0.8135
Epoch 0 step 88: training loss: 4230.616008118185
Epoch 0 step 89: training accuarcy: 0.8155
Epoch 0 step 89: training loss: 4449.043227141733
Epoch 0 step 90: training accuarcy: 0.8170000000000001
Epoch 0 step 90: training loss: 4342.627977375192
Epoch 0 step 91: training accuarcy: 0.8200000000000001
Epoch 0 step 91: training loss: 4437.580208013199
Epoch 0 step 92: training accuarcy: 0.8160000000000001
Epoch 0 step 92: training loss: 4352.9991128314705
Epoch 0 step 93: training accuarcy: 0.8255
Epoch 0 step 93: training loss: 4303.712833765702
Epoch 0 step 94: training accuarcy: 0.8210000000000001
Epoch 0 step 94: training loss: 4373.448334461702
Epoch 0 step 95: training accuarcy: 0.8200000000000001
Epoch 0 step 95: training loss: 4736.407048960332
Epoch 0 step 96: training accuarcy: 0.799
Epoch 0 step 96: training loss: 4413.52130234032

Epoch 0 step 171: training loss: 3278.014820346404
Epoch 0 step 172: training accuarcy: 0.857
Epoch 0 step 172: training loss: 3382.71176975627
Epoch 0 step 173: training accuarcy: 0.8565
Epoch 0 step 173: training loss: 3203.9518986471157
Epoch 0 step 174: training accuarcy: 0.8535
Epoch 0 step 174: training loss: 3306.1572768369983
Epoch 0 step 175: training accuarcy: 0.847
Epoch 0 step 175: training loss: 3400.3387793358434
Epoch 0 step 176: training accuarcy: 0.8525
Epoch 0 step 176: training loss: 3313.611008988354
Epoch 0 step 177: training accuarcy: 0.848
Epoch 0 step 177: training loss: 3192.588621227758
Epoch 0 step 178: training accuarcy: 0.8615
Epoch 0 step 178: training loss: 3325.320352853744
Epoch 0 step 179: training accuarcy: 0.856
Epoch 0 step 179: training loss: 3308.919915313704
Epoch 0 step 180: training accuarcy: 0.8515
Epoch 0 step 180: training loss: 3420.516076546224
Epoch 0 step 181: training accuarcy: 0.8575
Epoch 0 step 181: training loss: 3447.308250341687
E

Epoch 0 step 256: training loss: 2378.30446814462
Epoch 0 step 257: training accuarcy: 0.8865000000000001
Epoch 0 step 257: training loss: 2689.290625966659
Epoch 0 step 258: training accuarcy: 0.869
Epoch 0 step 258: training loss: 2599.7357488807675
Epoch 0 step 259: training accuarcy: 0.8715
Epoch 0 step 259: training loss: 2486.467270905074
Epoch 0 step 260: training accuarcy: 0.8765000000000001
Epoch 0 step 260: training loss: 2337.265990648815
Epoch 0 step 261: training accuarcy: 0.8805000000000001
Epoch 0 step 261: training loss: 2687.31482556325
Epoch 0 step 262: training accuarcy: 0.8685
Epoch 0 step 262: training loss: 1674.5314784903028
Epoch 0 step 263: training accuarcy: 0.867948717948718
Epoch 0: train loss 4187.308647550641, train accuarcy 0.7998396158218384
Epoch 0: valid loss 2786.230083246025, valid accuarcy 0.8596118688583374


...

Epoch 1 step 510: training loss: 1747.6074247200322
Epoch 1 step 511: training accuarcy: 0.9015
Epoch 1 step 511: training loss: 1660.8542156667604
Epoch 1 step 512: training accuarcy: 0.903
Epoch 1 step 512: training loss: 1661.4022571047935
Epoch 1 step 513: training accuarcy: 0.894
Epoch 1 step 513: training loss: 1684.4944126169858
Epoch 1 step 514: training accuarcy: 0.895
Epoch 1 step 514: training loss: 1584.5413906725576
Epoch 1 step 515: training accuarcy: 0.9
Epoch 1 step 515: training loss: 1562.1089164863388
Epoch 1 step 516: training accuarcy: 0.9055
Epoch 1 step 516: training loss: 1635.7283621700608
Epoch 1 step 517: training accuarcy: 0.902
Epoch 1 step 517: training loss: 1617.3579243219324
Epoch 1 step 518: training accuarcy: 0.9005
Epoch 1 step 518: training loss: 1650.9778458697035
Epoch 1 step 519: training accuarcy: 0.899
Epoch 1 step 519: training loss: 1705.6762999891944
Epoch 1 step 520: training accuarcy: 0.888
Epoch 1 step 520: training loss: 1776.61850285000

 50%|█████████████████████████████████████████████████████████████████████████                                                                         | 2/4 [03:54<03:56, 118.12s/it]

Epoch: 2
Epoch 2 step 526: training loss: 1646.548359780817
Epoch 2 step 527: training accuarcy: 0.897
Epoch 2 step 527: training loss: 1757.5727729512162
Epoch 2 step 528: training accuarcy: 0.902
Epoch 2 step 528: training loss: 1592.6025893717006
Epoch 2 step 529: training accuarcy: 0.9075
Epoch 2 step 529: training loss: 1564.6298620474167
Epoch 2 step 530: training accuarcy: 0.9085
Epoch 2 step 530: training loss: 1445.8536056388948
Epoch 2 step 531: training accuarcy: 0.9165
Epoch 2 step 531: training loss: 1588.1101836552687
Epoch 2 step 532: training accuarcy: 0.9015
Epoch 2 step 532: training loss: 1532.0823423229117
Epoch 2 step 533: training accuarcy: 0.911
Epoch 2 step 533: training loss: 1592.49515958836
Epoch 2 step 534: training accuarcy: 0.9005
Epoch 2 step 534: training loss: 1550.0078764846326
Epoch 2 step 535: training accuarcy: 0.901
Epoch 2 step 535: training loss: 1622.256007048045
Epoch 2 step 536: training accuarcy: 0.902
Epoch 2 step 536: training loss: 1637.84

Epoch 2 step 611: training loss: 1538.910563896608
Epoch 2 step 612: training accuarcy: 0.8945000000000001
Epoch 2 step 612: training loss: 1544.243201069412
Epoch 2 step 613: training accuarcy: 0.9
Epoch 2 step 613: training loss: 1416.5144179518422
Epoch 2 step 614: training accuarcy: 0.9035
Epoch 2 step 614: training loss: 1429.4535641328469
Epoch 2 step 615: training accuarcy: 0.916
Epoch 2 step 615: training loss: 1521.069822201428
Epoch 2 step 616: training accuarcy: 0.9095
Epoch 2 step 616: training loss: 1513.5068027623174
Epoch 2 step 617: training accuarcy: 0.9005
Epoch 2 step 617: training loss: 1524.7943883140304
Epoch 2 step 618: training accuarcy: 0.905
Epoch 2 step 618: training loss: 1469.7991410242728
Epoch 2 step 619: training accuarcy: 0.912
Epoch 2 step 619: training loss: 1528.3083228949506
Epoch 2 step 620: training accuarcy: 0.9065
Epoch 2 step 620: training loss: 1632.6855200553243
Epoch 2 step 621: training accuarcy: 0.897
Epoch 2 step 621: training loss: 1552.

Epoch 2 step 697: training accuarcy: 0.8945000000000001
Epoch 2 step 697: training loss: 1518.3340768774415
Epoch 2 step 698: training accuarcy: 0.899
Epoch 2 step 698: training loss: 1492.2449764692315
Epoch 2 step 699: training accuarcy: 0.905
Epoch 2 step 699: training loss: 1460.1931585250043
Epoch 2 step 700: training accuarcy: 0.9145
Epoch 2 step 700: training loss: 1545.656473456194
Epoch 2 step 701: training accuarcy: 0.9015
Epoch 2 step 701: training loss: 1467.9086901247329
Epoch 2 step 702: training accuarcy: 0.904
Epoch 2 step 702: training loss: 1491.3159299742822
Epoch 2 step 703: training accuarcy: 0.9095
Epoch 2 step 703: training loss: 1484.5099879280974
Epoch 2 step 704: training accuarcy: 0.9035
Epoch 2 step 704: training loss: 1510.3849245850133
Epoch 2 step 705: training accuarcy: 0.9055
Epoch 2 step 705: training loss: 1445.695599552168
Epoch 2 step 706: training accuarcy: 0.9035
Epoch 2 step 706: training loss: 1264.273495613737
Epoch 2 step 707: training accuarc

In [21]:
del prob_model
T.cuda.empty_cache()